# Uso de polymake: Trabajando con poliedros

## Representación de $\mathcal{V}$-poliedros

Internamente, `polymake` representa a los poliedros a través de sus conos de homogeneización. Conocemos que si $P = \mathrm{conv}(V) + \mathrm{cone}(Y) \subset \mathbb{R}^d$, entonces el cono de homogeneización de $P$ está dado por:
$$
\mathrm{homog}(P):= \mathrm{cone}\left(
\begin{array}{cc}
\mathbf{1}^T & \mathbf{0}^T\\
V & Y
\end{array}
\right) \subset \mathbb{R}^{d+1}.
$$

Los vectores (o puntos) en $\mathbb{R}^{d+1}$ para generar $\mathrm{homog}(P)$ se especifican por medio de la propiedad `POINTS`.

Por ejemplo, definimos a continuación `$p` como la envolvente convexa del conjunto $\{ (-1,-1), (-1,1), (1,-1), (1,1), (0,0) \}$:

In [ ]:
$p=new Polytope(POINTS=>[[1,-1,-1],[1,-1,1],[1,1,-1],[1,1,1],[1,0,0]]);
### mostrar las propiedades de $p
$p->properties;

Podemos obtener una representación gráfica de `$p` llamando al método `VISUAL`.

In [ ]:
### mostrar una representación gráfica de $p
$p->VISUAL;

Consultando la propiedad `VERTICES` podemos determinar cuáles son los puntos no redundantes requeridos para definir el polítopo (es decir, sus vértices):

In [ ]:
print $p->VERTICES;

La propiedad `DIM` nos devuelve la dimensión del polítopo:

In [ ]:
print $p->DIM;

La propiedad `FACETS` nos devuelve las desigualdades que definen las facetas del polítopo. Al consultarla, `polymake` invoca internamente un algoritmo tipo Fourier-Motzkin para transformar la representación $\mathcal{V}$ en la representación $\mathcal{H}$ del polítopo, a partir del cálculo de la envolvente convexa. Con la instrucción `prefer` podemos requerir específicamente que se utilice alguno de los métodos disponibles. Por ejemplo, en el siguiente fragmento de código se emplea el [algoritmo de búsqueda en reversa revisado `lrs`](http://cgm.cs.mcgill.ca/~avis/C/lrs.html) desarrollado por David Avis:

In [ ]:
prefer "lrs";
print_constraints($p->FACETS);

Con la propiedad `VERTICES_IN_FACETS` consultamos la incidencia entre vértices y facetas:

In [ ]:
print ($p->VERTICES_IN_FACETS);

Agregaremos ahora una parte cónica al poliedro anterior. Según la representación del cono de homogeinización, esto se consigue añadiendo un punto (o vector) con la primera coordenada igual a cero al especificar la propiedad `POINTS`.

En el ejemplo siguiente definimos a `$q` como $\mathrm{conv}(\{(-1,-1), (-1,1), (1,-1), (1,1), (0,0)\}) + \mathrm{cone}(\{(1,1)\})$

In [ ]:
$q = new Polytope(POINTS=>[[1,-1,-1],[1,-1,1],[1,1,-1],[1,1,1],[1,0,0],[0,1,1]]);
$q -> VISUAL;

Al consultar la propiedad `VERTICES` podemos ver que el nuevo poliedro tiene tres vértices (puntos no redundantes en la combinación convexa) y un rayo (vector no redundante en la combinación cónica). Los vértices tienen la primera coordenada igual a 1, los rayos tienen la primera coordenada igual a 0:

In [ ]:
print($q->VERTICES);

Podemos consultar también las desigualdades que definen las facetas de `$q`, así como la incidencia de facetas en vértices:

In [ ]:
### mostrar facetas de $q
print_constraints($q->FACETS);
print("---\n");

### mostrar incidencia de vértices en facetas
print ($q->VERTICES_IN_FACETS);

## Representación de $\mathcal{H}$-poliedros

Un $\mathcal{H}$-poliedro de la forma $P:= \{ x \in \mathbb{R}^d \, : \, Ax \leq b\}$ se representa a través se su cono de homogeinización, dado por el sistema de desigualdades:
$$
\left(
\begin{array}{cc}
1 & \mathbf{0}^T \\
b & -A
\end{array}
\right)
\left(
\begin{array}{c}
x_0 \\ x
\end{array}
\right)
\geq \mathbf{0}.
$$

Las filas de la matriz de coeficientes se especifican por medio de la propiedad `INEQUALITIES`; no es necesario incluir la primera fila, la cual es añadida automáticamente por polymake.

Por ejemplo, el siguiente fragmento de código asigna a la variable `$r` el poliedro en $\mathbb{R}^2$ definido por el sistema de desigualdades:
$$
\left\{
\begin{aligned}
& x_1 + x_2 \leq 4 \\
& x_1 + 2x_2 \leq 7 \\
& 0 \leq x_1 \leq 2 \\
& 0 \leq x_2 \leq 3 \\
\end{aligned}
\right.
$$

In [ ]:
$r = new Polytope(INEQUALITIES=>[[4,-1,-1],[7,-1,-2],[2,-1,0],[3,0,-1],[0,1,0], [0,0,1]]);
### mostrar las propiedades de $r
$r->properties;


Notar que polymake añadió automáticamente la fila `[1, 0, 0]`, correspondiente a la desigualdad $x_0 \geq 0$. 

La función `print_constraints` escribe el sistema de desigualdades en un formato más amigable para el usuario. Notar que esta función deshace además la homogeneización. En particular, la nonegatividad de $x_0$ se transforma en la desigualdad trivial $0 \geq -1$.

In [ ]:
print_constraints($r->INEQUALITIES);

Consultando la propiedad `FACETS` obtenemos las desigualdades no redundantes del sistema. En el ejemplo anterior, la segunda desigualdad no es listada, porque la misma se obtiene como suma de la primera y la cuarta desigualdades. De igual forma, la desigualdad trivial $0 \geq -1$ es eliminada del sistema.

In [ ]:
print_constraints($r->FACETS);

Con el método `VISUAL` obtenemos una representación gráfica de `$r`:

In [ ]:
$r->VISUAL;

La propiedad `VERTICES` nos indica los vértices y rayos necesarios para expresar al polítopo en la forma $\mathcal{V}$. Al consultar esta propiedad, `polymake`invoca automáticamente un algoritmo tipo Fourier-Motzkin para cambiar la representación del poliedro mediante la enumeración de sus vértices.

Recordar que los vértices tienen la primera coordenada igual a 1, mientras que los rayos tienen la primera coordenada igual a 0.

In [ ]:
### mostrar vértices y rayos de $r
print($r->VERTICES);

## Sumas de Minkowski

Definamos al polítopo `$p` como la envolvente convexa del conjunto de puntos $\{(1,1), (2,1), (1,2)\}$:

In [ ]:
$p=new Polytope(POINTS=>[[1,1,1],[1,2,1],[1,1,2]]);
$p->VISUAL;

Definamos ahora a `$q` como el cono generado por los vectores $10 \choose 9$ y $9 \choose 10$. Notar que es necesario indicar explícitamente el vértice del cono entre los elementos del parámetro `POINTS`
al llamar al constructor `Polytope`. Esto es requerido para obtener un cono de homogeneización de dimensión completa que pueda ser intersecado con el hiperplano $x_0 = 1$ para recuperar nuestro poliedro original.

In [ ]:
$q=new Polytope(POINTS=>[[1,0,0],[0,10,9],[0,9,10]]);
$q->VISUAL;

Para calcular la suma de Minkowski entre `$p` y `$q` podemos emplear la función `minkowski_sum`, la misma que retorna como resultado un nuevo poliedro:

In [ ]:
$m = minkowski_sum($p, $q);
$m -> VISUAL;

Podemos ahora consultar las diferentes propiedades del nuevo poliedro creado:

In [ ]:
print ($m->VERTICES);

In [ ]:
print_constraints($m->FACETS);

In [ ]:
print($m->VERTICES_IN_FACETS);

## Conos de recesión

Dado un poliedro, `polymake` puede emplearse para calcular su cono de recesión. Considerar el último poliedro `$m`que hemos definido:

In [ ]:
$m->VISUAL;

Para recuperar el cono de recesión de `$m` llamamos a la función `recession_cone`:

In [ ]:
$q2= recession_cone($m);
$q2 -> VISUAL;

Al contrario de lo que ocurre con los poliedros en general, este cono se representa sin homogeneización. La propiedad `RAYS` permite consultar los vectores que lo generan:

In [ ]:
print($q2->RAYS);
print("---\n");
print($q2->properties);
print("---\n");
print($q->properties);

## Ejercicio

1. Definamos un polítopo `$h` como un hexágono incrustado en el hiperplano $x_3=0$ en $\mathbb{R}^3$:

In [ ]:
### $h := conv({(1,0,0), (1/2,2/3,0), (-1/2,2/3,0), (-1,0,0), (1/2,-2/3,0), (-1/2,-2/3,0)})
$h=new Polytope(POINTS=>[[1,1,0,0],[1,1/2,2/3,0],[1,-1/2,2/3,0],[1,-1,0,0],
[1,1/2,-2/3,0],[1,-1/2,-2/3,0]]);
$h->VISUAL;

Mostremos las propiedades de `$h`:

In [ ]:
$h->properties;

Definamos ahora `$c` como el cono tridimensional generado por los vectores $\left( \begin{array}{c} 0 \\ 1 \\10 \end{array}\right)$, $\left( \begin{array}{c} 2/3 \\ -1/2 \\10 \end{array}\right)$ y $\left( \begin{array}{c} -2/3 \\ -1/2 \\10 \end{array}\right)$:

In [ ]:
### $c:= cone({(0,1,10), (2/3,-1/2,10), (-2/3,-1/2,10)})
$c=new Polytope(POINTS=>[[1,0,0,0],[0,0,1,10],[0,2/3,-1/2,10],[0,-2/3,-1/2,10]]);
$c->VISUAL;

Definamos `$p` como la suma de Minkowski de `$h` y `$c`:

In [ ]:
$p=minkowski_sum($h,$c);
$p->VISUAL;

Consultemos las desigualdades que definen las facetas de `$p`:

In [ ]:
print_constraints($p->FACETS);

Consultemos los vértices y rayos de `$p`:

In [ ]:
print($p->VERTICES);

Consultemos las incidencias entre los vértices, rayos y facetas de `$p`:

In [ ]:
print($p->VERTICES_IN_FACETS);

Recuperemos el cono de recesión de `$p`:

In [ ]:
$c2=recession_cone($p);
$c2->VISUAL;